In [1]:
import os
os.chdir("..")

from pprint import pprint
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from pprint import pprint

from tuxai.dataset import Dataset, Columns
from tuxai.misc import config_logger
from tuxai.report import model_metrics

config_logger()

## New method for correlated feature

In [2]:
df, groups = Dataset(508).get_dataframe(
    group_collinear_options=True,
    collinearity_threshold=0.0,
    return_collinear_groups=True
)
pprint(groups)
    

2022-12-01 11:57:50,559 :: INFO :: tuxai.featureselection :: group collinear options
2022-12-01 11:57:58,235 :: INFO :: tuxai.featureselection :: collinear options: 286 groups found, 509 columns removed


{'CORR|0000': ['EFI_STUB', 'EFI_GENERIC_STUB_INITRD_CMDLINE_LOADER'],
 'CORR|0001': ['EFI_DEV_PATH_PARSER', 'APPLE_PROPERTIES'],
 'CORR|0002': ['B44_PCICORE_AUTOSELECT', 'B44_PCI_AUTOSELECT', 'B44_PCI'],
 'CORR|0003': ['SSB_DRIVER_PCICORE_POSSIBLE', 'SSB_PCIHOST'],
 'CORR|0004': ['HAVE_ALIGNED_STRUCT_PAGE',
               'CC_OPTIMIZE_FOR_SIZE',
               'BITREVERSE',
               'KERNEL_GZIP',
               'REGMAP',
               'UNWINDER_GUESS',
               'CRC32_SLICEBY8',
               'SLUB',
               'HAVE_HARDENED_USERCOPY_ALLOCATOR',
               'KERNEL_XZ',
               'UNWINDER_ORC',
               'CC_OPTIMIZE_FOR_PERFORMANCE',
               'HAVE_RELIABLE_STACKTRACE',
               'SLOB'],
 'CORR|0005': ['QTNFMAC_PCIE', 'QTNFMAC'],
 'CORR|0006': ['ARCH_HAS_FORCE_DMA_UNENCRYPTED',
               'DYNAMIC_PHYSICAL_MASK',
               'AMD_MEM_ENCRYPT',
               'DMA_COHERENT_POOL'],
 'CORR|0007': ['WIMAX_I2400M_USB', 'WIMAX_I2400M'],
 

In [3]:
df['CORR|0006']

0        False
1        False
2        False
3         True
4        False
         ...  
21918    False
21919    False
21920     True
21921    False
21922     True
Name: CORR|0006, Length: 21923, dtype: bool